In [1]:
import os
import torch
import numpy as np
import random
from torch.autograd import Variable
from torch import nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image
#from mxtorch import transforms as tfs
import torchvision.transforms as tfs
from datetime import datetime
import six
import matplotlib.pyplot as plt

In [27]:
voc_root = '/Users/zhuqing/Documents/Github/Vision-for-Agriculture/Agriculture-Vision'
classes = ['cloud_shadow', 'double_plant', 'planter_skip', 'standing_water', 'waterway', 'weed_cluster']
colormap = [[0,0,0],[128,0,0],[0,128,0], [128,128,0], [0,0,128],
            [128,0,128],[0,128,128]]
def read_images(root=voc_root, train=True):
    if train:
        rgbdir = os.path.join(root, 'train','images','rgb')
        labeldir = os.path.join(root,'train','processedlabels')
        rgb_fig_names = os.listdir(rgbdir)
        fig_ids = [fname[:-4] for fname in rgb_fig_names]
        rgb_img = [os.path.join(rgbdir, fig_id + '.jpg') for fig_id in fig_ids]
        label_img=[os.path.join(labeldir, fig_id + '.png') for fig_id in fig_ids]
        return rgb_img, label_img
    else:
        rgbdir = os.path.join(root, 'val', 'images', 'rgb')
        labeldir = os.path.join(root,'val','processedlabels')
        rgb_fig_names = os.listdir(rgbdir)
        fig_ids = [fname[:-4] for fname in rgb_fig_names]
        rgb_img = [os.path.join(rgbdir, fig_id + '.jpg') for fig_id in fig_ids]
        label_img=[os.path.join(labeldir, fig_id + '.png') for fig_id in fig_ids]
        return rgb_img, label_img

def img_transforms(img):
    img_tfs = tfs.Compose([
        tfs.ToTensor(),
        tfs.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

    img = img_tfs(img)
    #labels= np.array(label, dtype='int64')
    #labels = torch.from_numpy(labels)
    return img

class VOCSegDataset(Dataset):
    '''
    voc dataset
    '''
    def __init__(self, train, transforms):
        #self.crop_size = crop_size
        self.transforms = transforms
        data_list, label_list = read_images(train=train)
        self.data_list = data_list
        self.label_list = label_list
        print('Read ' + str(len(self.data_list)) + ' images')

    def __getitem__(self, idx):
        img = self.data_list[idx]
        #label = self.label_list[idx]
        img = Image.open(img)
        #labels = Image.open(label)
        #img, labels = self.transforms(img, labels, self.crop_size)
        img = self.transforms(img)
        return img

    def __len__(self):
        return len(self.data_list)

def bilinear_kernel(in_channels, out_channels, kernel_size):
    '''
    return a bilinear filter tensor
    '''
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = np.ogrid[:kernel_size, :kernel_size]
    filt = (1 - abs(og[0] - center) / factor) * (1 - abs(og[1] - center) / factor)
    weight = np.zeros((in_channels, out_channels, kernel_size, kernel_size), dtype='float32')
    weight[range(in_channels), range(out_channels), :, :] = filt
    return torch.from_numpy(weight)

import torchvision.models as models
pretrained_net = models.resnet34(pretrained=True)

num_classes = 7

class fcn(nn.Module):
    def __init__(self, num_classes):
        super(fcn, self).__init__()

        self.stage1 = nn.Sequential(*list(pretrained_net.children())[:-4]) # 第一段
        self.stage2 = list(pretrained_net.children())[-4] # 第二段
        self.stage3 = list(pretrained_net.children())[-3] # 第三段

        self.scores1 = nn.Conv2d(512, num_classes, 1)
        self.scores2 = nn.Conv2d(256, num_classes, 1)
        self.scores3 = nn.Conv2d(128, num_classes, 1)

        self.upsample_8x = nn.ConvTranspose2d(num_classes, num_classes, 16, 8, 4, bias=False)
        self.upsample_8x.weight.data = bilinear_kernel(num_classes, num_classes, 16) # 使用双线性 kernel

        self.upsample_4x = nn.ConvTranspose2d(num_classes, num_classes, 4, 2, 1, bias=False)
        self.upsample_4x.weight.data = bilinear_kernel(num_classes, num_classes, 4) # 使用双线性 kernel

        self.upsample_2x = nn.ConvTranspose2d(num_classes, num_classes, 4, 2, 1, bias=False)
        self.upsample_2x.weight.data = bilinear_kernel(num_classes, num_classes, 4) # 使用双线性 kernel


    def forward(self, x):
        x = self.stage1(x)
        s1 = x # 1/8

        x = self.stage2(x)
        s2 = x # 1/16

        x = self.stage3(x)
        s3 = x # 1/32

        s3 = self.scores1(s3)
        s3 = self.upsample_2x(s3)
        s2 = self.scores2(s2)
        s2 = s2 + s3

        s1 = self.scores3(s1)
        s2 = self.upsample_4x(s2)
        s = s1 + s2

        s = self.upsample_8x(s)
        return s

net = fcn(num_classes)
net.load_state_dict(torch.load('/Users/zhuqing/Documents/Github/Vision-for-Agriculture/M/resnet34_mask.pth',map_location ='cpu'))
net = net.eval()
#net = net.cuda()
cm = np.array(colormap).astype('uint8')

def predict(img): # 预测结果
    img = Variable(img.unsqueeze(0))
    out = net(img)
    pred = out.max(1)[1].squeeze().cpu().data.numpy()
    pred = cm[pred]
    return pred

processdir="/Users/zhuqing/Desktop/Val_res"
data_list, label_list = read_images(voc_root,train=False)

for i in range(len(data_list)):
    img = data_list[i]
    img = Image.open(img)
    img = img_transforms(img)
    pred = predict(img)
    pred = np.array(pred, dtype='uint8')
    Im = Image.fromarray(pred)
    print(data_list[i][89:-4])
    outf = os.path.join(processdir, data_list[i][89:-4] + '.png')
    Im.save(outf, "PNG", quality=100)


IIIHV17VB_1193-750-1705-1262
IIIHV17VB_897-608-1409-1120
B87ABMJUV_4949-3274-5461-3786
J3WARFL9G_7489-794-8001-1306
X11EQAQ2H_1693-3875-2205-4387
7QEGPGTTI_5098-8695-5610-9207
KNRMZVPE2_6725-6285-7237-6797
L42DPYA11_1045-5603-1557-6115
J9PK1M3HW_1398-4475-1910-4987
7E6VFGT98_4396-3478-4908-3990
TCK29494G_6620-10465-7132-10977
KNRMZVPE2_6883-4667-7395-5179
W6YN2HWKX_1918-11227-2430-11739
Y3RA9WAPY_6496-15418-7008-15930
7E6VFGT98_2239-5918-2751-6430
ED7MDE861_595-928-1107-1440
QPVJVXYXQ_705-1136-1217-1648
XC3KHCKJ6_8901-2647-9413-3159
7E6VFGT98_1948-2343-2460-2855
QU2LAXW4R_3237-1295-3749-1807
J9PK1M3HW_2714-7222-3226-7734
QEQNXLHLY_7917-4184-8429-4696
UJ32E81HK_1296-11077-1808-11589
XC3KHCKJ6_14021-1111-14533-1623
BZNAQUMPX_3252-2700-3764-3212
M2JV8T29W_12547-9677-13059-10189
KNRMZVPE2_5701-6797-6213-7309
QP7AQX3RX_1115-1335-1627-1847
VGAED6RA4_666-5940-1178-6452
7QEGPGTTI_5046-6414-5558-6926
X11EQAQ2H_669-11043-1181-11555
ANG9VVZ6W_3716-3698-4228-4210
ZKV2W31CE_2435-957-2947-1469
2Z3HK

TYFTC8GV7_4562-5454-5074-5966
2Z3HK7FB7_5550-2755-6062-3267
X11EQAQ2H_2205-11555-2717-12067
QPVJVXYXQ_975-547-1487-1059
6GYIVDR9X_7547-4639-8059-5151
UJ32E81HK_1608-11012-2120-11524
KA33A8ZZY_5603-6763-6115-7275
7E6VFGT98_2981-3002-3493-3514
7HV64L31C_1355-15824-1867-16336
2Z3HK7FB7_7851-2658-8363-3170
L42DPYA11_952-6765-1464-7277
RNXV646CT_631-3172-1143-3684
VM41WRAYA_4054-4284-4566-4796
83XFQHLCL_6005-2793-6517-3305
NJ6YX7LE2_3644-6026-4156-6538
QEQNXLHLY_2204-3433-2716-3945
72U2692IY_9112-800-9624-1312
72U2692IY_23383-824-23895-1336
KNRMZVPE2_7777-7430-8289-7942
KNRMZVPE2_1519-1655-2031-2167
ZKV2W31CE_3575-1662-4087-2174
QP7AQX3RX_2292-2394-2804-2906
QEQNXLHLY_8429-3160-8941-3672
Y3RA9WAPY_2126-3151-2638-3663
BUX6LYPUV_2942-8562-3454-9074
B87ABMJUV_1781-2991-2293-3503
ZGGUZC42K_2678-2667-3190-3179
QEQNXLHLY_8599-3389-9111-3901
Y1KIN6HMD_2751-2271-3263-2783
BUX6LYPUV_4155-8545-4667-9057
2Z3HK7FB7_844-4863-1356-5375
83XFQHLCL_1208-7093-1720-7605
8XY6ZAPU4_2495-13149-3007-13661
PZKD88Y

88RUIM9H6_4604-5477-5116-5989
YZ3PPIIMD_3840-1127-4352-1639
TYFTC8GV7_6079-4845-6591-5357
LQ484RKYT_2360-1712-2872-2224
ED7MDE861_1056-2886-1568-3398
7HV64L31C_10293-1993-10805-2505
8Z42DJRIP_1087-1265-1599-1777
ED7MDE861_6562-4143-7074-4655
G2HM1BK7J_3596-1812-4108-2324
TCK29494G_7132-8417-7644-8929
I2MKNR9L4_2707-2123-3219-2635
IBYE8WX8Z_6914-872-7426-1384
QEQNXLHLY_8611-2127-9123-2639
LYZEMUHNQ_1992-904-2504-1416
KNRMZVPE2_3055-1143-3567-1655
Y3RA9WAPY_1612-6148-2124-6660
XC3KHCKJ6_10437-5719-10949-6231
B87ABMJUV_3317-3503-3829-4015
B87ABMJUV_4272-486-4784-998
QU2LAXW4R_5727-376-6239-888
ED7MDE861_1038-1554-1550-2066
88RUIM9H6_4187-2652-4699-3164
18X67NLZ2_1219-1603-1731-2115
8XY6ZAPU4_823-14706-1335-15218
ZKV2W31CE_2702-1085-3214-1597
EVIRB92AD_4166-8046-4678-8558
88RUIM9H6_1184-5422-1696-5934
VM41WRAYA_3369-7387-3881-7899
RYGD82W2F_1701-2014-2213-2526
86LYFYC79_1918-680-2430-1192
2Z3HK7FB7_8348-2427-8860-2939
VM41WRAYA_2544-4399-3056-4911
7HV64L31C_2272-15902-2784-16414
YJ9RN6WDX_

IEAFIMCIZ_1663-1505-2175-2017
83XFQHLCL_6005-2281-6517-2793
83XFQHLCL_4789-8234-5301-8746
C4QPKAKIG_1767-1896-2279-2408
83XFQHLCL_2152-786-2664-1298
QU2LAXW4R_5215-376-5727-888
ED7MDE861_3998-4842-4510-5354
QEQNXLHLY_1196-2684-1708-3196
IVA9BMCZ4_3256-6770-3768-7282
QP7AQX3RX_4812-2426-5324-2938
ZKV2W31CE_3335-4260-3847-4772
B87ABMJUV_6594-4018-7106-4530
HH1L2XBC2_2298-2608-2810-3120
4VUQI7PW6_1064-1328-1576-1840
MJWDUMQT9_9795-605-10307-1117
B6HTKJXHY_4005-1173-4517-1685
7QEGPGTTI_4593-1145-5105-1657
7HV64L31C_2565-390-3077-902
72U2692IY_866-4597-1378-5109
P1KBWN9YF_7555-4604-8067-5116
4VUQI7PW6_1198-2133-1710-2645
TCK29494G_709-15992-1221-16504
9JWQPBYH3_5718-8093-6230-8605
7QEGPGTTI_140-9746-652-10258
8XY6ZAPU4_5824-16381-6336-16893
72U2692IY_16608-8100-17120-8612
72U2692IY_14503-8083-15015-8595
ED7MDE861_3163-1453-3675-1965
X11EQAQ2H_3229-8995-3741-9507
TCK29494G_7132-5857-7644-6369
X11EQAQ2H_2717-12067-3229-12579
9JWQPBYH3_7595-8131-8107-8643
IIIHV17VB_1369-1533-1881-2045
KFWJ3VQY

ED7MDE861_6238-4517-6750-5029
TCK29494G_5596-4833-6108-5345
M2JV8T29W_10340-1986-10852-2498
QP7AQX3RX_4492-2802-5004-3314
4VUQI7PW6_1813-1429-2325-1941
ED7MDE861_788-3553-1300-4065
2Z3HK7FB7_875-5766-1387-6278
MJWDUMQT9_7209-3375-7721-3887
YR3V9IYY8_1327-623-1839-1135
Y3RA9WAPY_3747-134-4259-646
9Z9GP8D4A_1587-6741-2099-7253
2Z3HK7FB7_2508-8212-3020-8724
Y3RA9WAPY_16276-15291-16788-15803
LYZEMUHNQ_1920-2859-2432-3371
G2HM1BK7J_891-5410-1403-5922
83XFQHLCL_1483-2377-1995-2889
KFWJ3VQY1_5603-5416-6115-5928
QP7AQX3RX_5990-536-6502-1048
UJ32E81HK_1550-13496-2062-14008
BUX6LYPUV_2981-1704-3493-2216
NJ6YX7LE2_5453-7608-5965-8120
QEQNXLHLY_6978-2208-7490-2720
ANG9VVZ6W_1802-3330-2314-3842
C4QPKAKIG_1094-4058-1606-4570
2Z3HK7FB7_4680-3681-5192-4193
KJJJU3ICG_8240-3669-8752-4181
KJJJU3ICG_994-3666-1506-4178
7HV64L31C_11261-13106-11773-13618
UJ32E81HK_2402-10697-2914-11209
TCK29494G_3036-10977-3548-11489
2Z3HK7FB7_8359-1622-8871-2134
LQ484RKYT_795-6867-1307-7379
6GYIVDR9X_1219-6618-1731-7130
QEQ

...

LCGMXFR8K_4638-564-5150-1076
J9PK1M3HW_2502-7666-3014-8178
7E6VFGT98_4396-3990-4908-4502
G2HM1BK7J_3715-3165-4227-3677
G2HM1BK7J_4381-623-4893-1135
7QEGPGTTI_4837-5723-5349-6235
7E6VFGT98_1588-4328-2100-4840
I2MKNR9L4_1948-4537-2460-5049
6ZNXWNPB7_597-3929-1109-4441
9Z9GP8D4A_994-3080-1506-3592
88RUIM9H6_3930-7567-4442-8079
2Z3HK7FB7_2508-7700-3020-8212
A88IZM6PD_5446-3496-5958-4008
B87ABMJUV_5296-3046-5808-3558
72U2692IY_21166-8102-21678-8614
9JWQPBYH3_2777-8000-3289-8512
B87ABMJUV_5223-3914-5735-4426
KJJJU3ICG_691-1542-1203-2054
HIAGFE38Q_1694-2637-2206-3149
2Z6V32VW4_3289-863-3801-1375
YJ9RN6WDX_642-1253-1154-1765
JZREINH1B_2643-993-3155-1505
HH1L2XBC2_1014-2153-1526-2665
TYFTC8GV7_5598-2859-6110-3371
A88IZM6PD_4934-4008-5446-4520
9Z9GP8D4A_3107-6516-3619-7028
7HV64L31C_3851-6063-4363-6575
2Z9UV9DHA_2107-4663-2619-5175
QEQNXLHLY_2360-2544-2872-3056
2Z3HK7FB7_5170-3952-5682-4464
BUX6LYPUV_3279-547-3791-1059
YJ9RN6WDX_4500-3320-5012-3832
ED7MDE861_7022-3156-7534-3668
ED7MDE861_937-871

2Z3HK7FB7_1579-7031-2091-7543
IIIHV17VB_2144-3640-2656-4152
G2HM1BK7J_1979-3685-2491-4197
QP7AQX3RX_8299-667-8811-1179
B87ABMJUV_3760-2534-4272-3046
2Z3HK7FB7_2680-5817-3192-6329
9VE81Y1VW_2150-8167-2662-8679
LMNRQUGTX_1238-977-1750-1489
GACTBITC8_5905-3280-6417-3792
ANG9VVZ6W_1248-4020-1760-4532
2Z3HK7FB7_4250-2090-4762-2602
YJ9RN6WDX_3889-4007-4401-4519
LCGMXFR8K_4947-592-5459-1104
DPRTGMCWE_3249-866-3761-1378
2Z3HK7FB7_1768-478-2280-990
NX2MCGMWA_4415-3761-4927-4273
QEQNXLHLY_6597-2108-7109-2620
88RUIM9H6_865-6024-1377-6536
8XY6ZAPU4_2673-12820-3185-13332
NJ6YX7LE2_4463-5603-4975-6115
PPFDIQ81U_4702-3713-5214-4225
J9PK1M3HW_3014-6642-3526-7154
QP7AQX3RX_3542-2059-4054-2571
I2MKNR9L4_2155-2465-2667-2977
QEQNXLHLY_2393-3903-2905-4415
2Z9UV9DHA_2107-5175-2619-5687
A88IZM6PD_6084-3342-6596-3854
HIAGFE38Q_1925-2522-2437-3034
2Z3HK7FB7_3086-6075-3598-6587
6GYIVDR9X_4518-2289-5030-2801
Y3RA9WAPY_2902-13196-3414-13708
ANG9VVZ6W_4506-748-5018-1260
HIAGFE38Q_3423-887-3935-1399
G2HM1BK7J_860-3

C4QPKAKIG_4876-3547-5388-4059
72U2692IY_866-4085-1378-4597
IIIHV17VB_3614-2536-4126-3048
HH1L2XBC2_4346-3632-4858-4144
LMNRQUGTX_2262-2513-2774-3025
LYZEMUHNQ_3967-2031-4479-2543
ZKV2W31CE_1471-2571-1983-3083
ED7MDE861_5629-747-6141-1259
NEB8BZCTE_3838-3984-4350-4496
VM41WRAYA_1868-6738-2380-7250
8Z42DJRIP_1106-620-1618-1132
QU2LAXW4R_4703-376-5215-888
GEUHUK6MU_2787-2809-3299-3321
B87ABMJUV_7856-486-8368-998
UJ32E81HK_11299-13678-11811-14190
KJJJU3ICG_8534-2283-9046-2795
BZNAQUMPX_2740-3724-3252-4236
KA33A8ZZY_5603-4715-6115-5227
QEQNXLHLY_1692-3644-2204-4156
Q8YAZVDVV_1544-3392-2056-3904
X11EQAQ2H_4765-9507-5277-10019
6XBJAAQIC_6716-4779-7228-5291
72U2692IY_5688-820-6200-1332
88RUIM9H6_1147-4398-1659-4910
7QEGPGTTI_6301-9116-6813-9628
NEB8BZCTE_3742-4796-4254-5308
Y3RA9WAPY_4558-15458-5070-15970
UJ32E81HK_1687-13793-2199-14305
BUX6LYPUV_1956-8521-2468-9033
7E6VFGT98_655-3023-1167-3535
NX2MCGMWA_4875-5358-5387-5870
ZKV2W31CE_3959-3027-4471-3539
HH1L2XBC2_2298-3120-2810-3632
ZGGUZC42K_

ANG9VVZ6W_2657-2949-3169-3461
NJ6YX7LE2_4557-7326-5069-7838
UJ32E81HK_3609-10733-4121-11245
HH1L2XBC2_3811-1878-4323-2390
ZKV2W31CE_4023-2419-4535-2931
BZNAQUMPX_2490-2971-3002-3483
83XFQHLCL_5112-8204-5624-8716
Q3MENL8VD_10143-2683-10655-3195
7HV64L31C_10805-6601-11317-7113
HIAGFE38Q_1478-2452-1990-2964
TYFTC8GV7_4355-1474-4867-1986
G9N8LNVPV_1137-794-1649-1306
LQ484RKYT_2355-3478-2867-3990
ED7MDE861_6000-2668-6512-3180
9Z9GP8D4A_1587-6229-2099-6741
IIIHV17VB_4638-1512-5150-2024
2Z3HK7FB7_6242-4883-6754-5395
PC9UAQ8ZM_3066-566-3578-1078
ZKV2W31CE_3297-2251-3809-2763
3FJMLWRT2_1466-3026-1978-3538
UJ32E81HK_2102-11173-2614-11685
7E6VFGT98_1679-3023-2191-3535
ZGGUZC42K_2966-3100-3478-3612
UJ32E81HK_2886-9445-3398-9957
QP7AQX3RX_2910-1966-3422-2478
LQ484RKYT_1758-1468-2270-1980
6GYIVDR9X_2288-3511-2800-4023
2Z9UV9DHA_756-7172-1268-7684
YJ9RN6WDX_1931-672-2443-1184
M2JV8T29W_6893-896-7405-1408
XC3KHCKJ6_10949-4695-11461-5207
83XFQHLCL_6604-2411-7116-2923
JM89QM16V_5565-3343-6077-3855
J3WAR

J9PK1M3HW_3291-7938-3803-8450
ED7MDE861_1950-2282-2462-2794
VM41WRAYA_3773-3273-4285-3785
TCK29494G_2245-15480-2757-15992
F1JV84NKD_1178-1397-1690-1909
4VUQI7PW6_733-4598-1245-5110
HIAGFE38Q_5809-2393-6321-2905
72U2692IY_21847-824-22359-1336
VM41WRAYA_2013-5005-2525-5517
F1JV84NKD_675-2302-1187-2814
W6YN2HWKX_1217-12071-1729-12583
Y3RA9WAPY_2614-11493-3126-12005
VM41WRAYA_3565-7221-4077-7733
TU4111V1D_3882-5851-4394-6363
TU4111V1D_3721-6742-4233-7254
DPRTGMCWE_2599-4065-3111-4577
ANG9VVZ6W_4647-557-5159-1069
6GYIVDR9X_3686-3779-4198-4291
LQ484RKYT_5877-8391-6389-8903
ED7MDE861_5366-1588-5878-2100
B87ABMJUV_1200-2534-1712-3046
B87ABMJUV_5296-486-5808-998
ED7MDE861_2500-832-3012-1344
6GYIVDR9X_5030-2289-5542-2801
QEQNXLHLY_3250-3118-3762-3630
XC3KHCKJ6_9377-7834-9889-8346
BZNAQUMPX_4398-5706-4910-6218
ANG9VVZ6W_1472-2278-1984-2790
KNRMZVPE2_1519-631-2031-1143
2Z3HK7FB7_4200-2535-4712-3047
Q3MENL8VD_4734-2859-5246-3371
L42DPYA11_2549-3497-3061-4009
G2APW2Z2Z_723-4204-1235-4716
LYZEMUHNQ_2